In [1]:
# urlretrieve : url로 표시된 자원(이미지, 영상, 음성, ...)을 로컬PC로 복사
from urllib.request import urlretrieve

In [5]:
url = 'https://search.pstatic.net/common/?src=http%3A%2F%2Fcafefiles.naver.net%2FMjAyMDA0MTJfMTg3%2FMDAxNTg2NjY2MDY3NDgx.kDPPZZHDfdLwSAcPKj-vvE2Gw3vXzlgmjo-TmvrPLpUg.R9DAoPz8NnaPB6eiPNNlMseg1Ntnont4vDVCMzWu8lAg.JPEG%2FexternalFile.jpg&type=a340'

urlretrieve(url, 'images/apple.png')

('images/apple.png', <http.client.HTTPMessage at 0x20a291a53d0>)

## 네이버 이미지 주소 수집

In [6]:
import requests as req
from bs4 import BeautifulSoup as bs
from urllib.request import urlretrieve
from tqdm.notebook import tqdm as tn

# 웹 페이지를 요청/응답받는 함수
def getRequest(url):
    header = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'
    }
    return req.get(url, headers=header)

# 응답받은 데이터를 파이썬 객체로 변환하는 함수
def transform(url):
    return bs(getRequest(url).text, 'html.parser')

In [12]:
url = 'https://www.bing.com/images/search?q=%eb%b0%b0&form=HDRSC2&first=1&tsc=ImageBasicHover'

html = transform(url)

In [28]:
src_list = []

for img in html.select('img.mimg'):
    # 요소의 속성값 접근
    
    # src 속성이 없는 경우
    if img.get('src') == None:
#         print(img.get('data-src'))
        src_list.append(img.get('data-src'))
    else:
        # src 속성값에 'data:'가 포함된 경우(수집 못하는 이미지)
        if 'data' in img.get('src'):
            continue
        else:
#             print(img.get('src'))
            src_list.append(img.get('src'))
#     print('*'*50)

In [29]:
len(src_list)

25

In [32]:
# 이미지 주소를 활용하여 로컬PC에 'images' 폴더로 저장

for i in range(len(src_list)):
    urlretrieve(src_list[i], 'images/{}_{}.png'.format('pear', i+1))

## 확장자 추출

In [35]:
img_src = 'https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMTAxMTFfMjUg%2FMDAxNjEwMzY0MTk3NjA5.FT6ng6TZilc_5W9mIpmXwR6lA0EcmkkMX77rD0_1fGwg.9I9oF_HpOinSWUC9dp67jBYc8tCWs67C1eJctpH7pDog.JPEG.choionzoo%2FIMG_9565.jpg&type=a340'

# rfind() : 해당 문자열의 마지막 발생 인덱스를 반환
start = img_src.rfind('.')+1
end = img_src.rfind('&')

img_src[start:end]

'jpg'

## 네이버 상영작 & 예정작 영화 포스터 수집

In [36]:
url = 'https://movie.naver.com/movie/running/current.nhn'

html = transform(url)

In [61]:
for img in tn(html.select('div.thumb img')):
    img_src = img.get('src')
    img_name = img.get('alt').strip()
    
    start = img_src.rfind('.')+1
    end = img_src.rfind('?')
    
    urlretrieve(img_src, 'movie/{}.{}'.format(img_name.replace(": ", '_'), img_src[start:end]))